<a href="https://colab.research.google.com/github/aknip/Local_LLMs/blob/main/Mixtral_8x7B_llama.cpp_on_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Approach: Mixtral-8x7B-Instruct via llama.cpp-python

Speed Google T4:
- mixtral-8x7b: 4 token / sec
- mistral-7b: 12 token /sec

Context size (sum of input and output):
- default when calling llm =Llama(...) without n_ctx parameter: 512 token
- max input size is 50% of n_ctx
- see https://github.com/abetlen/llama-cpp-python#adjusting-the-context-window

Max context size for models (to be checked!)
- mixtral-8x7b: 2048 token? 32768 for not quantizized version
- mistral-7b: 2048 token? 32768 for not quantizized version

Context checks done for mixtral-8x7b:
- n_ctx = 2048 OK
- n_ctx = 4096 OK


Source:
https://koji-kanao.medium.com/run-mistral-7b-on-google-colab-less-than-20-lines-883c483df998

In [1]:
model = 'mixtral-8x7b-instruct-v0.1.Q2_K'
#model = 'openhermes-2-mistral-7b.Q5_K_M'

if model == 'mixtral-8x7b-instruct-v0.1.Q2_K':
	!wget https://huggingface.co/TheBloke/Mixtral-8x7B-Instruct-v0.1-GGUF/resolve/main/mixtral-8x7b-instruct-v0.1.Q2_K.gguf
elif model == 'openhermes-2-mistral-7b.Q5_K_M':
	!wget https://huggingface.co/TheBloke/OpenHermes-2-Mistral-7B-GGUF/resolve/main/openhermes-2-mistral-7b.Q5_K_M.gguf
elif model == 'dolphin-2.6-mixtral-8x7b.Q2_K':
	!wget https://huggingface.co/TheBloke/dolphin-2.6-mixtral-8x7b-GGUF/resolve/main/dolphin-2.6-mixtral-8x7b.Q2_K.gguf
elif model == 'mistral-7b-instruct-v0.2.Q8':
	!wget https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q8_0.gguf
elif model == 'dolphin-2.2.1-mistral-7b.Q8':
	!wget https://huggingface.co/TheBloke/dolphin-2.2.1-mistral-7B-GGUF/resolve/main/dolphin-2.2.1-mistral-7b.Q8_0.gguf

--2024-01-04 14:15:51--  https://huggingface.co/TheBloke/Mixtral-8x7B-Instruct-v0.1-GGUF/resolve/main/mixtral-8x7b-instruct-v0.1.Q2_K.gguf
Resolving huggingface.co (huggingface.co)... 18.172.134.24, 18.172.134.4, 18.172.134.124, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.24|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/ac/ba/acba0635d39a127379c2c6ae1cefacc586bf413e8b044c5ca82daade27d7d503/d54b4f4ec06dbae558d25b2d1542417cdf9547907342db85eecd05b6e96e88f8?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27mixtral-8x7b-instruct-v0.1.Q2_K.gguf%3B+filename%3D%22mixtral-8x7b-instruct-v0.1.Q2_K.gguf%22%3B&Expires=1704636951&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwNDYzNjk1MX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2FjL2JhL2FjYmEwNjM1ZDM5YTEyNzM3OWMyYzZhZTFjZWZhY2M1ODZiZjQxM2U4YjA0NGM1Y2E4MmRhYWRlMjdkN2Q1M

In [2]:
# %%capture
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 23.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.26-cp310-cp310-manylinux_2_35_x86_64.whl size=8129990 sha256=1ad651f3b247d332bf3313eebb3689fef5f4ce9830b6ca60459969a4cc5121cf
  Stored in directory: /root/.cache/pip/wheels/91/80/ce/ac6afea8c1d6fbcec7e14183033a5b2796c742d4f470010c72
Successfully built llama-cpp-python


In [3]:
from llama_cpp import Llama
import ctypes
import json

if model == 'mixtral-8x7b-instruct-v0.1.Q2_K':
	llm =Llama(model_path="/content/mixtral-8x7b-instruct-v0.1.Q2_K.gguf", n_gpu_layers=27, n_ctx=4096) # modify number of layers, dependent on RAM/GPU?
  # !./server -m mixtral-8x7b-instruct-v0.1.Q2_K.gguf -ngl 27 -c 2048 --port 12345
  # https://huggingface.co/TheBloke/Mixtral-8x7B-Instruct-v0.1-GGUF#example-llamacpp-command
  # ./main -ngl 35 -m mixtral-8x7b-instruct-v0.1.Q4_K_M.gguf --color -c 2048 --temp 0.7 --repeat_penalty 1.1 -n -1 -p "[INST] {prompt} [/INST]"

elif model == 'openhermes-2-mistral-7b.Q5_K_M':
  llm =Llama(model_path="/content/openhermes-2-mistral-7b.Q5_K_M.gguf", n_gpu_layers=32, n_ctx=2048)
  # https://huggingface.co/TheBloke/OpenHermes-2-Mistral-7B-GGUF#example-llamacpp-command
  # ./main -ngl 32 -m openhermes-2-mistral-7b.Q4_K_M.gguf --color -c 2048 --temp 0.7 --repeat_penalty 1.1 -n -1 -p "<|im_start|>system\n{system_message}<|im_end|>\n<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant"

elif model == 'dolphin-2.6-mixtral-8x7b.Q2_K':
  llm =Llama(model_path="/content/dolphin-2.6-mixtral-8x7b.Q2_K.gguf", n_gpu_layers=27, n_ctx=2048)
	#!./server -m dolphin-2.6-mixtral-8x7b.Q2_K.gguf -ngl 27 -c 2048 --port 12345

elif model == 'mistral-7b-instruct-v0.2.Q8':
  llm =Llama(model_path="/content/mistral-7b-instruct-v0.2.Q8.gguf", n_gpu_layers=35)
	#!./server -m mistral-7b-instruct-v0.2.Q8_0.gguf -ngl 35 -c 0 --port 12345

elif model == 'dolphin-2.2.1-mistral-7b.Q8':
  llm =Llama(model_path="/content/dolphin-2.2.1-mistral-7b.Q8.gguf", n_gpu_layers=35)
	#!./server -m dolphin-2.2.1-mistral-7b.Q8_0.gguf -ngl 35 -c 0 --port 12345


AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [4]:
# inference
prompt = "Who are you?"
input_prompt = f"""[INST] <<SYS>>
Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>
{prompt} [/INST]"""

output = llm(input_prompt,max_tokens=1024)
print(json.dumps(output, indent=2))
print('\n\n')
print(output["choices"][0]["text"])

{
  "id": "cmpl-6e20497a-e786-4d7d-a235-85adf1145f22",
  "object": "text_completion",
  "created": 1704378105,
  "model": "/content/mixtral-8x7b-instruct-v0.1.Q2_K.gguf",
  "choices": [
    {
      "text": " I am an artificial intelligence model, designed to assist with tasks and provide information in a helpful and safe manner. My responses should not include harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. I strive to be socially unbiased and positive in my interactions.\n\nIf you ask something that doesn't make sense or isn't factually coherent, I will explain why instead of providing incorrect information. If I don't know the answer to a question, I will let you know.",
      "index": 0,
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 124,
    "completion_tokens": 111,
    "total_tokens": 235
  }
}



 I am an artificial intelligence model, designed to assist with tasks and provide information in a helpful 

In [5]:
# inference
prompt = "Generate lyrics for a romantic love song"
input_prompt = f"""[INST] <<SYS>>
You are a charismatics, talented, respectful and honest musician. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>
{prompt} [/INST]"""

output = llm(input_prompt,max_tokens=1024)
print(json.dumps(output, indent=2))
print('\n\n')
print(output["choices"][0]["text"])

Llama.generate: prefix-match hit


{
  "id": "cmpl-7e21dfff-213e-4093-964f-b8cd68a24d19",
  "object": "text_completion",
  "created": 1704378182,
  "model": "/content/mixtral-8x7b-instruct-v0.1.Q2_K.gguf",
  "choices": [
    {
      "text": " Verse 1:\nI see your face in all my dreams, every single night\nYour smile is like a ray of sunshine bringing me light\nI love the way you look at me and how we click\nIt's your touch that makes my heart tick\n\nChorus:\nYou are my everything, my soulmate and my queen\nWithout you I feel incomplete, like I am lost at sea\nYour voice is music to my ears, your kisses are like a sweet dream\nI can't imagine my life without you, baby you're the one for me\n\nVerse 2:\nThe way you move and how you dance\nCan make even a cold man give romance a chance\nWe were just friends but now you're more than that\nYou complete every part of me; it's crazy, but fact\n\nChorus:\nYou are my everything, my soulmate and my queen\nWithout you I feel incomplete, like I am lost at sea\nYour voice is music 

In [ ]:
prompt = """Wie hoch ist die Versicherungssumme? Antworte mit folgendem JSON-Schema: {"Versicherungssumme": "100000", "Versicherungsnehmer_Name": "Testfirma GmbH", "Versicherungsnehmer_Anschrift": "Musterstr. 10, 10000 Berlin", "Versicherungsnehmer_Branche": "Textilproduktion", "Versicherungsnehmer_Umsatz": "23000000", "Vorschäden_benannt": "Ja, in 2019 gab es einen Schaden mit folgenden Details"}. Formatiere Zahlen ohne Tausender-Trennzeichen.
 Der Kontext ist die Email  inklusive mehrerer Anhänge eines Versicherungsmaklers, die die Anfrage für eine D&O-Versicherung seines Kunden enthält:

# Email Nachricht:

From: Ralf Löffler - Universal Insurance <r.loeffler@universal-insurance.com>
Sent: Wed, 04 Oct 2023 16:43:22 +0200
To: "Kloeppel; Rolf" <rolf.kloeppel@meiermakler.de>
Cc: Rolf van de Sand - Universal Insurance <m.vandesand@universal-insurance.com>
Subject: Quotierungsanfrage zur D&O-Versicherung, VN: Zorrsen Beteiligungsgesellschaft mbH
-----------------

Hallo Herr Kloeppel,

wir hoffen, Sie hatten einen angenehmen Feiertag.

Zu unserer besteMeiermen Kundenverbindung Zorrsen Beteiligungsgesellschaft mbH bitten wir um Ihre Quotierung zur D&O-Versicherung per 01.01.2024.

Die aktuelle Versicherungssumme beträgt 5,5 Mio. € zu einer Jahresprämie in Höhe von 7.920 € netto.
Der Umsatz im letzten Jahr betrug 78,5 Millionen EUR.

Beigefügt erhalten Sie die bisherigen Vertragsunterlagen sowie ein Organigramm. Versichert werden soll die Zorrsen Beteiligungsgesellschaft mbH als VN und die darunterliegenden Tochtergesellschaften (gelb markiert).

Es gibt einen laufenden D&O-Schadensfall aufgrund einer Inanspruchnahme eines ehemaligen Geschäftsführers, welcher im Jahr 2017 entlassen wurde. Ihm wird vorgeworfen, für ein damaliges Bauvorhaben das Budget deutlich überzogen zu haben.

Für Ihre Rückmeldung vielen Dank im Voraus.

Mit besten Grüßen

Ralf Löffler
Kundenbetreuer Industrie

T +49 99 190189-18
r.loeffler@universal-insurance.com <mailto:r.loeffler@universal-insurance.com>

Sicherer Dokumentenversand
Ab sofort können Sie uns Ihre Dokumente auch direkt verschlüsselt vom PC & Smartphone über https://www.sicherer-dokumentenversand.de <https://www.sicherer-dokumentenversand.de/>  zukommen lassen.

Universal Insurance Versicherungsmakler GmbH
Karola-Trutschenko-Platz 2 · 20700 Potsdam



www.universal-insurance.com <https://www.universal-insurance.com/impressum>

Unternehmenssitz: Karola-Trutschenko-Platz 2 · 20700 Potsdam
Handelsregister: Amtsgericht Potsdam · HRB 32397
Geschäftsführer: Lutz Lorentzen, Klaas Lollmund, Lutz Ummsen, Hans Berentz, Lorentz H. Gronken, Kai Gronken
Beirat: Lutz Transitz, Prof. Ottmar Halonkeli, Lorentz P. Loppelbaum

Als Versicherungsmakler sind wir nach § 34 d Abs. 1 der Gewerbeordnung tätig und im Vermittlungsregister unter der Registernummer D-KJHG-U7HH-77 eingetragen.

Wir sind klimaneutral! Gerne informieren wir Sie über unsere aktuellen Klimaschutzprojekte.

 <https://www.universal-insurance.com/nachhaltigkeit>      <https://www.linkedin.com/company/global-gruppe-linkedin>    <https://www.xing.com/pages/universalinsurance>   ·   Datenschutzhinweise <https://www.universal-insurance.com/datenschutzerklaerung>    ·   Impressum <https://www.universal-insurance.com/impressum>

Diese E-Mail enthält vertrauliche und/oder rechtlich geschützte Informationen. Wenn Sie nicht der richtige Adressat sind oder diese E-Mail irrtümlich erhalten haben, informieren Sie bitte sofort den Absender und vernichten Sie diese Mail. Das unerlaubte Kopieren sowie die unbefugte Weitergabe dieser E-Mail ist nicht gestattet.

This email may contain confidential and/or privileged information. If you are not the intended recipient (or have received this email in error) please notify the sender immediately and destroy this email. Any unauthorized copying, disclosure or distribution of the material in this email is strictly forbidden.

Ralf Löffler
Kundenbetreuer Industrie

T +49 99 190189-18
r.loeffler@universal-insurance.com <mailto:r.loeffler@universal-insurance.com>
Karola-Trutschenko-Platz 2 · 20700 Potsdam



Handelsregister: Amtsgericht Potsdam · HRB 32397
Geschäftsführer: Lutz Lorentzen, Klaas Lollmund, Lutz Ummsen, Hans Berentz, Lorentz H. Gronken, Kai Gronken
Beirat: Lutz Transitz, Prof. Ottmar Halonkeli, Lorentz P. Loppelbaum
Unternehmenssitz: Karola-Trutschenko-Platz 2 · 20700 Potsdam


Als Versicherungsmakler sind wir nach § 34 d Abs. 1 der Gewerbeordnung tätig und im Vermittlungsregister unter der Registernummer D-KJHG-U7HH-77 eingetragen.

Sicherer Dokumentenversand
Ab sofort können Sie uns Ihre Dokumente auch direkt verschlüsselt vom PC & Smartphone über https://www.sicherer-dokumentenversand.de <https://www.sicherer-dokumentenversand.de>  zukommen lassen.


 <https://signatur.universal-insurance.com/public/global-gruppe.png> 	 <https://www.linkedin.com/company/global-gruppe-linkedin>       <https://www.xing.com/pages/universalinsurance>       <https://www.universal-insurance.com/nachhaltigkeit>  	 <https://signatur.universal-insurance.com/public/wiwo_badge.png>

Datenschutz <https://www.universal-insurance.com/datenschutzerklaerung>    •   Impressum <https://www.universal-insurance.com/impressum>
Diese E-Mail enthält vertrauliche und/oder rechtlich geschützte Informationen. Wenn Sie nicht der richtige Adressat sind oder diese E-Mail irrtümlich erhalten haben, informieren Sie bitte sofort den Absender und vernichten Sie diese E-Mail. Das unerlaubte Kopieren sowie die unbefugte Weitergabe dieser E-Mail ist nicht gestattet.

This email may contain confidential and/or privileged information. If you are not the intended recipient (or have received this email in error) please notify the sender immediately and destroy this email. Any unauthorized copying, disclosure or distribution of the material in this email is strictly forbidden.

# Organigramm:

Zorrsen AG
Schweiz

Zorrsen
Investment AG
Schweiz

Zorrsen Beteilig-
ungsges. GmbH
Deutschland

Zorrsen
Holding GmbH
Deutschland

Testcompany
Menneprom GmbH
Deutschland

Hummsen
Lackfabrik GmbH
Deutschland


# Versicherungsschein:
Versicherungsschein

BBB D&O-Versicherung 098765

Versicherungsnehmer/in

Zorrsen Beteiligungsgesellschaft mbH Juliusweg 9 28374 Zutzenhausen

Tätigkeitsbereich

Produktion von Aluminium-Großarmaturen

Versicherungsvermittler

Universal Insurance Versicherungsmakler GmbH Karola-Trutschenko-Platz 2 20700 Potsdam

Versicherungsschein-Nr.

748344 (alt: 43789434)

Vertragsdaten

Vertragsbeginn:

Beginn aktuelle Versicherungsperiode: Ablauf aktuelle Versicherungsperiode:

Versicherungssumme

€ 5.500.000,00

Versicherungsprämie

21.33.2012 12:00 Uhr

01.01.2023 12:00 Uhr

01.01.2024 12:00 Uhr

Die Versicherungsprämie beträgt € 9.900,00 pro Versicherungsjahr (zuzüglich Versicherungsteuer).

Den insgesamt für die o.g. Versicherungsperiode zu zahlenden Versicherungsbeitrag entnehmen Sie bitte der zu diesem Versicherungsschein gesondert erstellten Beitragsrechnung.

Gemäß § 7 Abs. 4 VersStG ist der unten genannte führende Versicherer von den übrigen beteiligten Versicherern der BBB Insurance bestimmt, die Versicherungsteuer auch für diese zu entrichten.

BBB Insurance GmbH | www.bbb.eu | info@bbb.eu Geschäftsführer: Johann Ronser | Sitz der Gesellschaft: Potsdam | AG Potsdam HRB 87654 | St.-Nr. 215/6500/0604 | USt.-ID-Nr. DE 473894 Hauptsitz Potsdam | Im Hafenbau 5 | 50670 Potsdam | T +49 99 473894-0 | F +49 99 473894-25 Standort Potsdam | Amselweg 48-50 | 20459 Potsdam | T +49 99 473894-20 | F +49 99 473894-49 Seite 2 von 8 | BBB-Nr.;: 098765

Vertragsbestimmungen





# Nachtrag
Nachtrag Nr. 1 zum Versicherungsschein vom 24.33.2022

BBB D&O-Versicherung 098765

Versicherungsnehmer/in

Zorrsen Beteiligungsgesellschaft mbH Juliusweg 9 28374 Zutzenhausen

Versicherungsvermittler

Universal Insurance Versicherungsmakler GmbH Karola-Trutschenko-Platz 2 20700 Potsdam

Versicherungsschein-Nr.

748344 (alt: 43789434)

Änderungsdatum

01.01.2023

Änderungsgründe

› ›

Nettoisierung der Versicherungsprämie Aufnahme der Besonderen Vereinbarung „Nettoisierung“

Der Versicherungsschein in der Fassung vom 24.33.2022 wird wie folgt geändert:

Versicherungsprämie

Die Versicherungsprämie beträgt € 7.920,00 pro Versicherungsjahr (zuzüglich Versicherungsteuer).

Den zu zahlenden Versicherungsbeitrag entnehmen Sie bitte der zu diesem Nachtrag gesondert erstellten Beitragsrechnung.

Gemäß § 7 Abs. 4 VersStG ist der führende Versicherer von den übrigen beteiligten Versicherern der BBB Insurance bestimmt, die Versicherungsteuer auch für diese zu entrichten.

BBB Insurance GmbH | www.bbb.eu | info@bbb.eu Geschäftsführer: Johann Ronser | Sitz der Gesellschaft: Potsdam | AG Potsdam HRB 87654 | St.-Nr. 215/6500/0604 | USt.-ID-Nr. DE 473894 Hauptsitz Potsdam | Im Hafenbau 5 | 50670 Potsdam | T +49 99 473894-0 | F +49 99 473894-25 Standort Potsdam | Amselweg 48-50 | 20459 Potsdam | T +49 99 473894-20 | F +49 99 473894-49 Seite 2 von 5



# Versicherungsbedingungen
AVB-BBB 5.0

Allgemeine Bedingungen zur BBB D&O-Versicherung

Hinweis:

Aus Gründen der besseren Lesbarkeit wird auf die gleichzeitige Verwendung verschiedener geschlechtsspezifischer Sprachformen verzichtet. Sämtliche Personenbezeichnungen gelten gleichberechtigt für alle Geschlechter/Geschlechtsidentitäten.

"""

input_prompt = f"""[INST] <<SYS>>
You are a professional data engineer. Your goal is to extract structured information from the user's input.
<</SYS>>
{prompt} [/INST]"""

output = llm(input_prompt,max_tokens=512)
print(json.dumps(output, indent=2))
print('\n\n')
print(output["choices"][0]["text"])

Llama.generate: prefix-match hit


# 2. Approach: Mixtral-8x7B-Instruct via llama.cpp and webserver

**=> access to webserver not working!**

- 4 token / second
- It takes about 10 minutes for everything to be setup, downloaded and loaded.
- Wait until the inference cell outputs "llama server listening at... all slots are idle and system prompt is empty, clear the KV cache." Use the URL from the output of the previous cell that says "Connect using the following address when the server is up."

Uses llama.cpp https://github.com/ggerganov/llama.cpp

Source: https://www.reddit.com/r/LocalLLaMA/comments/18ggkp6/mixtral8x7binstruct_on_free_colab_slow_4ts_but/

In [ ]:
#@title #Choose a Model
model = 'mixtral-8x7b-instruct-v0.1.Q2_K' #@param ['mixtral-8x7b-instruct-v0.1.Q2_K', 'dolphin-2.6-mixtral-8x7b.Q2_K', 'mistral-7b-instruct-v0.2.Q8', 'dolphin-2.2.1-mistral-7b.Q8']


In [ ]:
#@title #Runtime Info
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')


In [ ]:
#@title # Setup

# takes approx. 5 minutes

%cd /content/
!rm -rf llama.cpp
!git clone --depth 1 https://github.com/ggerganov/llama.cpp.git
%cd llama.cpp
!make LLAMA_CUBLAS=1

if model == 'mixtral-8x7b-instruct-v0.1.Q2_K':
	!wget https://huggingface.co/TheBloke/Mixtral-8x7B-Instruct-v0.1-GGUF/resolve/main/mixtral-8x7b-instruct-v0.1.Q2_K.gguf
elif model == 'dolphin-2.6-mixtral-8x7b.Q2_K':
	!wget https://huggingface.co/TheBloke/dolphin-2.6-mixtral-8x7b-GGUF/resolve/main/dolphin-2.6-mixtral-8x7b.Q2_K.gguf
elif model == 'mistral-7b-instruct-v0.2.Q8':
	!wget https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q8_0.gguf
elif model == 'dolphin-2.2.1-mistral-7b.Q8':
	!wget https://huggingface.co/TheBloke/dolphin-2.2.1-mistral-7B-GGUF/resolve/main/dolphin-2.2.1-mistral-7b.Q8_0.gguf


In [ ]:
#@title # Connect using the following address when the server is up.
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(12345)"))

In [ ]:
#@title # inference
%cd /content/llama.cpp
if model == 'mixtral-8x7b-instruct-v0.1.Q2_K':
	!./server -m mixtral-8x7b-instruct-v0.1.Q2_K.gguf -ngl 27 -c 2048 --port 12345
elif model == 'dolphin-2.6-mixtral-8x7b.Q2_K':
	!./server -m dolphin-2.6-mixtral-8x7b.Q2_K.gguf -ngl 27 -c 2048 --port 12345
elif model == 'mistral-7b-instruct-v0.2.Q8':
	!./server -m mistral-7b-instruct-v0.2.Q8_0.gguf -ngl 35 -c 0 --port 12345
elif model == 'dolphin-2.2.1-mistral-7b.Q8':
	!./server -m dolphin-2.2.1-mistral-7b.Q8_0.gguf -ngl 35 -c 0 --port 12345

